In [1]:
import re
import pandas as pd 

In [2]:
f = open('WhatsApp Chat with CSPC_PMDU.txt', 'r', encoding='utf-8')
data=f.read()

In [3]:
pattern = '\d{1,2}/\d{1,2}/\d{1,2},\s\d{1,2}:\d{2}\s\w{2}\s-\s'
messages = re.split(pattern, data)[1:]
len(messages)

3427

In [4]:
dates=re.findall(pattern, data)
len(dates)

3427

In [5]:
df= pd.DataFrame({'user_massage':messages, 'massage_date':dates})

In [6]:
# convert message_date type
df['massage_date']

0       5/14/19, 10:10 PM - 
1       5/14/19, 10:06 PM - 
2       5/14/19, 10:10 PM - 
3       5/14/19, 10:11 PM - 
4       5/14/19, 10:11 PM - 
                ...         
3422      3/4/24, 1:13 PM - 
3423      3/4/24, 6:34 PM - 
3424      3/5/24, 1:57 PM - 
3425     3/6/24, 10:51 AM - 
3426     3/6/24, 11:35 AM - 
Name: massage_date, Length: 3427, dtype: object

In [7]:
df['massage_date'] = df['massage_date'].str.replace("-","").str.strip()
df['massage_date'] = df['massage_date'].str.replace("/",".")
df['massage_date'] = df['massage_date'].str.replace(",","")
df['massage_date'] = pd.to_datetime(df['massage_date'])

In [8]:
df.rename(columns={'massage_date':'date'}, inplace=True)

In [9]:
users=[]
massages=[]
for massage in df['user_massage']:
    entry = re.split('([\w\W]+?):\s', massage)
    if entry[1:]: #user_name
        users.append(entry[1])
        massages.append(" ".join(entry[2:]))
    else:
        users.append('group_notification')
        massages.append(entry[0])

In [10]:
df['users'] = users
df['massages'] = massages

In [11]:
df['only_date'] = df['date'].dt.date


In [12]:
df['year'] = df['date'].dt.year


In [13]:
df['month_name']=df['date'].dt.month_name()


In [14]:
df['month_num'] = df['date'].dt.month


In [15]:
df['day'] = df['date'].dt.day

In [16]:
df['day'] = df['date'].dt.day_name()


In [17]:
df['hour']= df['date'].dt.hour


In [18]:
df['minute']= df['date'].dt.minute


In [19]:
df['second']= df['date'].dt.second


In [20]:
user_list = df['users'].unique().tolist()
user_list.remove('group_notification')
user_list.sort()

In [21]:
user_list.insert(0,"Overall")

In [22]:
df['massages'].shape[0]

3427

In [23]:
len(df['massages'])

3427

In [24]:
words = []
for massage in df['massages']:
    words.extend(massage.split())

In [25]:
len(words)

36903

In [26]:
num_media_messages = df[df['massages'] == '<Media omitted>\n']

In [27]:
len(num_media_messages)

2307

In [28]:
from urlextract import URLExtract
extract = URLExtract()
num_links = []
for massage in df['massages']:
    words.extend(extract.find_urls(massage))

ModuleNotFoundError: No module named 'urlextract'

In [29]:
x=df['users'].value_counts().head()
Busy_users_df = round((df['users'].value_counts() / df.shape[0]) * 100, 2).reset_index().rename(columns={'index' : 'User_name', 'users' : 'Percent'})

In [30]:
f = open('stop_hinglish.txt', 'r')
stop_words = f.read()

In [31]:
temp = df[df['users'] != 'group_notification']

In [32]:
temp = temp[temp['massages'] != '<Media omitted>\n']

In [33]:
def remove_stop_words(massage):
    y = []
    for word in massage.lower().split():
        if word not in stop_words:
            y.append(word)
    return " ".join(y)

In [34]:
temp['massages'] = temp['massages'].apply(remove_stop_words)

In [ ]:
from wordcloud import WordCloud
wc = WordCloud()
df_wc = wc.generate(temp['massages'].str.cat(sep=" "))


In [ ]:
import matplotlib.pyplot as plt
fig,ax = plt.subplots()
#ax.imshow(df_wc)

In [ ]:
def common_words(massage):
    y = []
    for word in massage.lower().split():
        if word not in stop_words:
            y.append(word)
    return y
words = temp['massages'].apply(common_words)

In [36]:
busy_day = df['day'].value_counts().reset_index().rename(columns={'index':'day','day':'massages'})

In [58]:
daily_timeline = df.groupby(['only_date']).count()['massages'].reset_index()

In [57]:
busy_month=df['month_name'].value_counts()

In [55]:
montly_timeline = df.groupby(['year', 'month_num', 'month_name']).count()['massages'].reset_index() #reset_index() convert it to DataFrame
time = []
for i in range (montly_timeline.shape[0]):
    time.append(montly_timeline['month_name'][i] + " " + str(montly_timeline['year'][i]))

montly_timeline ['time']  = time

In [59]:
df.head()

,user_massage,date,users,massages,only_date,year,month_name,month_num,day,hour,minute,second
0,Messages and calls are end-to-end encrypted. N...,2019-05-14 22:10:00,group_notification,Messages and calls are end-to-end encrypted. N...,2019-05-14,2019,May,5,Tuesday,22,10,0
1,"Ahmad Sb CSPC created group ""Office""\n",2019-05-14 22:06:00,group_notification,"Ahmad Sb CSPC created group ""Office""\n",2019-05-14,2019,May,5,Tuesday,22,6,0
2,Ahmad Sb CSPC added you\n,2019-05-14 22:10:00,group_notification,Ahmad Sb CSPC added you\n,2019-05-14,2019,May,5,Tuesday,22,10,0
3,Azam DEO ITBP 2: Asslamoalaikum sir\n,2019-05-14 22:11:00,Azam DEO ITBP 2,Asslamoalaikum sir\n,2019-05-14,2019,May,5,Tuesday,22,11,0
4,Azam DEO ITBP 2: Good idea for group\n,2019-05-14 22:11:00,Azam DEO ITBP 2,Good idea for group\n,2019-05-14,2019,May,5,Tuesday,22,11,0


In [64]:
daily_timeline=df.groupby(['only_date']).count()['massages'].reset_index()

In [75]:
period = []
for hour in df[['day', 'hour']]['hour']:
    if hour == 23:
        period.append(str(hour) + "-" + str('00'))
    elif hour == 0:
        period.append(str('00') + "-" + str(hour + 1))
    else:
        period.append(str(hour) + "-" + str(hour + 1))

df['period'] = period

In [79]:
user_heatmap = df.pivot_table(index='day', columns='period', values='massages', aggfunc='count').fillna(0)
user_heatmap

period,00-1,1-2,10-11,11-12,12-13,13-14,14-15,15-16,16-17,17-18,...,21-22,22-23,23-00,3-4,4-5,5-6,6-7,7-8,8-9,9-10
day,,,,,,,,,,,,,,,,,,,,,
Friday,14.0,1.0,20.0,25.0,75.0,60.0,24.0,104.0,33.0,25.0,...,7.0,58.0,31.0,6.0,8.0,2.0,4.0,13.0,5.0,44.0
Monday,4.0,3.0,14.0,20.0,18.0,18.0,79.0,84.0,15.0,13.0,...,4.0,54.0,11.0,3.0,0.0,1.0,3.0,6.0,6.0,48.0
Saturday,4.0,3.0,7.0,14.0,25.0,15.0,42.0,12.0,19.0,7.0,...,43.0,13.0,130.0,4.0,4.0,8.0,1.0,10.0,15.0,10.0
Sunday,13.0,87.0,64.0,13.0,15.0,53.0,11.0,9.0,18.0,16.0,...,28.0,25.0,19.0,4.0,2.0,1.0,3.0,3.0,4.0,6.0
Thursday,5.0,1.0,27.0,22.0,17.0,31.0,10.0,31.0,25.0,14.0,...,25.0,23.0,14.0,4.0,4.0,1.0,5.0,5.0,8.0,20.0
Tuesday,6.0,1.0,13.0,27.0,10.0,10.0,44.0,40.0,18.0,15.0,...,38.0,36.0,21.0,10.0,1.0,1.0,2.0,8.0,5.0,12.0
Wednesday,4.0,30.0,42.0,26.0,20.0,15.0,35.0,43.0,83.0,28.0,...,45.0,17.0,52.0,2.0,2.0,1.0,5.0,21.0,12.0,50.0
